In [15]:
import numpy as np
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# Khai báo trình duyệt Chrome
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Mở URL
driver.get("https://vnexpress.net/goc-nhin")
driver.maximize_window()

def scroll_page():
    driver.execute_script("window.scrollBy(0, -100);")  # Scroll up slightly
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(random.uniform(2, 4))

def close_popups():
    try:
        popup = driver.find_element(By.CSS_SELECTOR, "selector-for-popup-close-button")
        popup.click()
        sleep(1)
    except NoSuchElementException:
        pass  # No pop-up found, continue

def click_and_wait(link, retries=3):
    """
    Điều hướng đến liên kết bài viết.
    """
    for attempt in range(retries):
        try:
            # Điều hướng đến liên kết
            driver.get(link)
            sleep(2)
            return
        except Exception as e:
            print(f"Retrying navigation, attempt {attempt + 1}: {e}")
            sleep(1)
    print("Failed to navigate to the link after retries.")


# Hàm để trích xuất thông tin thẻ HTML sau khi truy cập bài viết
def extract_detail_info():
    try:
        # Lấy thông tin "date"
        date_element = driver.find_element(By.CSS_SELECTOR, "span.date")
        date = date_element.text

        # Lấy thông tin "h1.title-detail"
        title_detail_element = driver.find_element(By.CSS_SELECTOR, "h1.title-detail")
        detailed_title = title_detail_element.text

        # Lấy thông tin "p.description"
        description_element = driver.find_element(By.CSS_SELECTOR, "p.description")
        detailed_description = description_element.text

        # Lấy thông tin nội dung bài viết
        content_paragraphs = driver.find_elements(By.CSS_SELECTOR, "article.fck_detail p")
        content = "\n".join([paragraph.text for paragraph in content_paragraphs])

        return {
            "Date": date,
            "Detailed Title": detailed_title,
            "Author's Position": detailed_description,
            "Content": content
        }
    except NoSuchElementException:
        print("Không tìm thấy thông tin chi tiết.")
        return {}


def extract_articles():
    data = []
    i = 0

    while i < 5:  # Dừng lại sau khi xử lý 5 bài viết
        try:
            # Lấy lại danh sách bài viết từ DOM mỗi lần lặp
            articles = driver.find_elements(By.CSS_SELECTOR, "article.item-news")
            
            # Kiểm tra nếu không còn bài viết để xử lý
            if i >= len(articles):
                break

            # Chọn bài viết theo chỉ số i
            article = articles[i]

            # Tiêu đề và link
            title_element = article.find_element(By.CSS_SELECTOR, "h3.title-news a")
            title = title_element.text
            print("Title found: ", title)
            link = title_element.get_attribute("href")
            print("Link found: ", link)

            # Mô tả
            description_element = article.find_element(By.CSS_SELECTOR, "p.description a")
            description = description_element.text

            # Số lượng comment
            try:
                comment_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.count_cmt span.font_icon")
                comments = comment_element.text.strip()
            except NoSuchElementException:
                comments = 0  # Mặc định không có bình luận


            # Tác giả
            author_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.name-author")
            author = author_element.text
            author_link = author_element.get_attribute("href")

            # Danh mục
            category_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.cat")
            category = category_element.text
            category_link = category_element.get_attribute("href")
            

            # Điều hướng đến liên kết bài viết
            click_and_wait(link)
            print("Navigated to the link.")

            # Lấy thông tin chi tiết từ bài viết
            detail_info = extract_detail_info()
            print("Detail info: ", detail_info)

            # Lưu dữ liệu
            data.append({
                "Title": title,
                "Link": link,
                "Description": description,
                "Author": author,
                "Author Link": author_link,
                "Category": category,
                "Category Link": category_link,
                "Comments": comments,
                **detail_info
            })

            # Trở lại trang chính
            driver.back()
            sleep(2)

            # Tăng chỉ số bài viết
            i += 1

        except NoSuchElementException:
            print("Không tìm thấy phần tử")
            i += 1
            continue
        except StaleElementReferenceException:
            print("Stale element detected, refreshing articles list.")
            continue

    return data



# Gọi hàm extract_articles và in kết quả
articles_data = extract_articles()
print(pd.DataFrame(articles_data))


Title found:  Visa quá cảnh ngặt nghèo
Link found:  https://vnexpress.net/visa-qua-canh-ngat-ngheo-4831587.html
Navigated to the link.
Detail info:  {'Date': 'Thứ tư, 25/12/2024, 00:00 (GMT+7)', 'Detailed Title': 'Visa quá cảnh ngặt nghèo', "Author's Position": 'Kỹ sư điều hành', 'Content': 'Những thay đổi này góp phần cải thiện rõ rệt chất lượng hoạt động xuất nhập cảnh và trải nghiệm của du khách đến Việt Nam. Tuy nhiên mới đây tôi nhận thấy một vấn đề còn tồn tại, tuy nhỏ nhưng rất cần sự thay đổi.\nDo yêu cầu công việc, tôi phải học thêm bộ môn mới là "Quản lý và điều hành sân bay". Thầy giáo của tôi trước khi về hưu là giám đốc điều hành một sân bay lớn ở Australia, từng tư vấn và thiết kế hệ thống thông tin, quản lý an toàn ở các sân bay trên khắp thế giới. Thấy tôi là người Việt, trong giờ giải lao, ông chia sẻ một trải nghiệm cá nhân.\nVợ chồng thầy đi du lịch nước Anh và đặt chuyến bay khứ hồi từ Australia quá cảnh Việt Nam với một hãng hàng không Việt Nam. Chuyến chiều đi hạn

In [17]:
# Lưu vào file Excel
articles_data = pd.DataFrame(articles_data)
articles_data.to_excel("D:/du lieu o cu/HUTECH Courses/Social Networking Course/SocialNetworkingProject/Project của Đạt/vnexpress_articles.xlsx")
